#  FHIR Bulk Inference from HAPI Server
Fetch real-world Observation data from [HAPI FHIR public test server](https://hapi.fhir.org/baseR4) and perform model inference.

ModuleNotFoundError: No module named 'fhir.resources.fhirdate'

In [53]:
# Sample Framingham-style data
data = {
    "age": [55, 60],
    "sysBP": [140, 135],
    "diaBP": [85, 90],
    "totChol": [240, 230],
    "glucose": [100, 110],
    "heartRate": [70, 72]
}
df = pd.DataFrame(data)

# LOINC mappings
loinc_map = {
    "sysBP": ("8480-6", "Systolic blood pressure", "mmHg"),
    "diaBP": ("8462-4", "Diastolic blood pressure", "mmHg"),
    "totChol": ("2093-3", "Total Cholesterol", "mg/dL"),
    "glucose": ("2345-7", "Glucose", "mg/dL"),
    "heartRate": ("8867-4", "Heart rate", "beats/minute"),
    "age": ("30525-0", "Age calculated", "years")
}

# Output folder
output_dir = "/mnt/data/fhir_sim/observations"
os.makedirs(output_dir, exist_ok=True)

In [54]:
# Generate fake FHIR Observation JSONs
for idx, row in df.iterrows():
    patient_id = f"Patient-{1000 + idx}"
    for feature, value in row.items():
        code, display, unit = loinc_map[feature]
        obs = Observation.construct()
        obs.id = f"{patient_id}-{feature}"
        obs.status = "final"
        obs.code = CodeableConcept.construct()
        obs.code.coding = [Coding.construct(system="http://loinc.org", code=code, display=display)]
        obs.subject = FHIRReference.construct(reference=patient_id)
        obs.effectiveDateTime = FHIRDate(datetime.utcnow().isoformat())
        obs.valueQuantity = Quantity.construct(value=value, unit=unit, system="http://unitsofmeasure.org", code=unit)
        obs.meta = Meta.construct(profile=["http://hl7.org/fhir/StructureDefinition/Observation"])

        # Save
        output_path = os.path.join(output_dir, f"{obs.id}.json")
        with open(output_path, "w") as f:
            f.write(obs.json(indent=2))

output_dir

C:\Users\tranv\AppData\Local\Temp\ipykernel_16720\3721412361.py:6: PydanticDeprecatedSince20: The `construct` method is deprecated; use `model_construct` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  obs = Observation.construct()


ValidationError: 1 validation error for Observation
status
  Value for the field 'status' is required. [type=model_field_validation.missing, input_value=None, input_type=NoneType]